# Trade full matrix

This notebook is to use in colab for a large dataset. To read the file as chunks, select some columns and save csv.

In [51]:
#!pip install ydata-profiling

In [19]:
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import zipfile
import io

## Read a sample of random rows from large file

In [28]:
"""
%%time

# Define the sample size
sample_size = 1000

# Specify the path to your zip file
zip_file_path = "/content/drive/MyDrive/NUCLIO_drive/TFM_drive/data/FAOSTAT/all_raw/Trade_DetailedTradeMatrix_E_All_Data_(Normalized).zip"
csv_file_name = "Trade_DetailedTradeMatrix_E_All_Data_(Normalized).csv"  # Name of the CSV inside the zip

# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as myzip:

    # Read the header of the CSV file
    with myzip.open(csv_file_name, 'r') as file:
        header = file.readline().decode('ISO-8859-1').strip()

    # Count the total number of rows in the CSV file (excluding header)
    with myzip.open(csv_file_name, 'r') as file:
        total_rows = sum(1 for row in file) - 1  # Count all lines except the header

    # Generate random row indices to include
    include_indices = set(np.random.choice(np.arange(1, total_rows + 1), size=sample_size, replace=False))

    # Read the CSV file and extract the desired rows
    with myzip.open(csv_file_name, 'r') as file:
        selected_rows = []
        for i, line in enumerate(file):
            if i == 0:
                continue  # Skip the header
            if i in include_indices:
                selected_rows.append(line.decode('ISO-8859-1').strip())

    # Create a DataFrame from the selected rows
    trade_mx_sample = pd.read_csv(io.StringIO('\n'.join([header] + selected_rows)), encoding="ISO-8859-1")

# Now trade_mx_sample contains a sample of the data
print(trade_mx_sample.head())
"""

   Reporter Country Code Reporter Country Code (M49)   Reporter Countries  \
0                      1                        '051              Armenia   
1                     14                        '052             Barbados   
2                      8                        '028  Antigua and Barbuda   
3                     11                        '040              Austria   
4                      2                        '004          Afghanistan   

   Partner Country Code Partner Country Code (M49) Partner Countries  \
0                    57                       '112           Belarus   
1                   255                       '056           Belgium   
2                    55                       '212          Dominica   
3                    10                       '036         Australia   
4                   113                       '417        Kyrgyzstan   

   Item Code Item Code (CPC)  \
0        114          '23180   
1       1232          'F1232   
2       

# Read the whole trading matrix and save only import data in quantities

In [1]:
import zipfile
import pandas as pd

# Specify the path to your zip file
zip_file_path = "/content/drive/MyDrive/NUCLIO_drive/TFM_drive/data/FAOSTAT/all_raw/Trade_DetailedTradeMatrix_E_All_Data_(Normalized).zip"
csv_file_name = "Trade_DetailedTradeMatrix_E_All_Data_(Normalized).csv"

# Initialize an empty dataframe to store the results
filtered_data = pd.DataFrame()

# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as myzip:
    # Directly read the CSV file using ZipFile.open
    with myzip.open(csv_file_name, 'r') as file:
        # Iterate over chunks of the file
        chunk_size = 100000  # Adjust the chunk size as needed
        for chunk in pd.read_csv(file, encoding="ISO-8859-1", chunksize=chunk_size):
            # Filter and drop columns
            chunk = chunk[chunk['Unit'] != '1000 USD']
            chunk = chunk[chunk['Element'] == 'Import Quantity']
            cols_todrop = ['Reporter Country Code (M49)', 'Partner Country Code (M49)', 'Year Code', 'Element', 'Element Code']
            chunk.drop(columns=cols_todrop, inplace=True)

            # Append the processed chunk to the filtered_data dataframe
            filtered_data = pd.concat([filtered_data, chunk], ignore_index=True)

# Save the filtered data to a new CSV file
filtered_data.to_csv('/content/drive/MyDrive/NUCLIO_drive/TFM_drive/data/FAOSTAT/trade_matrix_import_only.csv', index=False)


<ipython-input-1-4cc13a5dbea6>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk.drop(columns=cols_todrop, inplace=True)
<ipython-input-1-4cc13a5dbea6>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk.drop(columns=cols_todrop, inplace=True)
<ipython-input-1-4cc13a5dbea6>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk.drop(columns=cols_todrop, inplace=True)
<ipython-input-1-4cc13a5dbea6>:22: SettingWithCopyWarning:

In [2]:
df = pd.read_csv('/content/drive/MyDrive/NUCLIO_drive/TFM_drive/data/FAOSTAT/trade_matrix_import_only.csv')
df.head()

,Reporter Country Code,Reporter Countries,Partner Country Code,Partner Countries,Item Code,Item Code (CPC),Item,Year,Unit,Value,Flag
0,3,Albania,3,Albania,828,'25020.01,Cigarettes,2004,t,12.00,A
1,4,Algeria,2,Afghanistan,231,'21422,"Almonds, shelled",2005,t,3.00,A
2,3,Albania,3,Albania,56,'0112,Maize (corn),2005,t,52.00,A
3,7,Angola,2,Afghanistan,176,'01701,"Beans, dry",2022,t,25.06,A
4,4,Algeria,2,Afghanistan,711,'01654,"Anise, badian, coriander, cumin, caraway, fenn...",1996,t,171.00,A
